In [5]:
import csv
import sys
import random
import os
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import sparse
from scipy import linalg
import scipy.sparse.linalg as spla
from scipy.spatial import distance
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.mlab as mlab
from mpl_toolkits.mplot3d import axes3d
%matplotlib tk

In [6]:
sys.path.append("../Python_code")
from reddit import *
from path import *
from project_data_analysis import *
from generate_proj_to_remove import *

In [7]:
projects_to_remove = get_list_of_removed_proj(output_filename = "", writeto_file = False)

In [4]:
project_dict = dict()
with open('../data/atlas.json') as f:
    data = json.load(f)
projects_count = 0
for element in data['atlas']:
    pic_id = int(element["id"])
    if pic_id not in projects_to_remove:
        project_dict[pic_id] = projects_count
        projects_count += 1
print(projects_count)

1920


In [20]:
users_dict = dict()
users_count = 0
with open("../data/sorted_tile_placements_proj.csv",'r') as file:
    # Skip first line (header row)
    next(file, None)

    reader = csv.reader(file)
    
    for r in reader:
        user = r[1]
        if user not in users_dict:
            users_dict[user] = users_count
            users_count += 1

In [11]:

def distance_per_project(input_file_proj, projects_to_remove,sample_size):
    #input_file_proj= "../data/sorted_tile_placements_proj.csv"

    user_project_mat,users_dict,projects_dict = user_project_matrix_pixel_color(input_file_proj, projects_to_remove)

    proj_euc_dist = dict()  # Average euclidean distance between sampled users
    proj_cos_dist = dict()  # Average cosine distance between sampled users
    sample_size = 100

    users_per_proj=users_per_project_list(input_file_proj, projects_to_remove)

    for pic_id in users_per_proj:
        all_users = list(users_per_proj[pic_id]) # make a list
        if len(all_users) > sample_size: # sample from it
            all_users = random.sample(all_users, sample_size)

        count=0
        euc_dis=0.0
        cos_dis=0.0
        for i in range(len(all_users)):
            user1 = all_users[i]

            # If the user has no contributions that made it to the final canvas, then skip him
            if np.sum(user_project_mat[users_dict[user1], : ]) == 0:
                continue

            for j in range(i + 1, len(all_users)):

                # If the user has no contributions that made it to the final canvas, then skip him
                user2 = all_users[j]
                if np.sum(user_project_mat[users_dict[user2], : ]) == 0:
                    continue
                count=count+1
                vec1 = user_project_mat[users_dict[user1], : ].toarray()
                vec2 = user_project_mat[users_dict[user2], :].toarray()
                euc_dis = euc_dis+distance.euclidean( vec1 , vec2 )
                cos_dis = cos_dis+distance.cosine( vec1 , vec2 )   

        proj_euc_dist[pic_id]=euc_dis/count
        proj_cos_dist[pic_id]=cos_dis/count
    return proj_euc_dist, proj_cos_dist

Euc_dist_proj, Cos_dist_proj = distance_per_project("../data/sorted_tile_placements_proj.csv", projects_to_remove,100)


KeyboardInterrupt: 

In [ ]:
sorted_euc_dist_per_proj = sorted(Euc_dist_proj.items(), key=operator.itemgetter(1), reverse=True)
#Top-10
for i in range(10):
    proj = sorted_euc_dist_per_proj[i][0]
    n = sorted_euc_dist_per_proj[i][1]
    name = names[int(proj)]
    desc = descriptions[int(proj)]
    
    print("#", i, ", project: ", name, ", distance: ", n, ", \ndesc: ", desc, "\n")

In [22]:
%%time
"""
    Type1 matrix: position (i,j) is the number of pixels by user i in project j 
"""

# Syntax:
# euc_dist = distance.euclidean(a,b)
# cos_dist = distance.cosine(a,b)

"""
    user_euc_dist and user_cos_dist is a dictionary of the following format:
        pic_id: user_distance list
    Create a new list for every project where every element is the euclidean distance or
    cosine distance between two users in a project.
    
    user_distances is a dictionary that stores the distance between two user vectors so we do not have to recompute it.
"""

users_euc_dist = dict()  # Position (i,j) is the euclidean distance between user vectors i and j
users_cos_dist = dict()  # Position (i,j) is the cosine distance between user vectors i and j
user_distances = dict()

# Calculating distance between every pair of users takes too long. (Several years)
# Sample at most 100 users for each project
sample_size = 100

for pic_id in users_per_project:
    all_users = list(users_per_project.get(pic_id))
#     print(len(all_users))
    
    
    if len(all_users) > sample_size:
        all_users = random.sample(all_users, sample_size)

    users_euc_dist[pic_id] = []
    users_cos_dist[pic_id] = []
#     print(user_matrix[pic_id].shape())
    
    for i in range(len(all_users)):
        user1 = all_users[i]
        
        # If the user has no contributions that made it to the final canvas, then skip him
        if np.sum(type1[users_dict[user1], : ]) == 0:
            continue
        
        for j in range(i + 1, len(all_users)):
            
            # If the user has no contributions that made it to the final canvas, then skip him
            user2 = all_users[j]
            if np.sum(type1[users_dict[user2], : ]) == 0:
                continue
            
            
            # Make sure user1 is the smaller value
            if user2 < user1:
                temp = user2
                user2 = user1
                user1 = temp
                
            if (user1, user2) not in user_distances:
                
                vec1 = type1[users_dict[user1], : ].toarray()
                vec2 = type1[users_dict[user2], :].toarray()
                euc_dist = distance.euclidean( vec1 , vec2 )
                cos_dist = distance.cosine( vec1 , vec2 )

                user_distances[(user1, user2)] = (euc_dist, cos_dist)

            users_euc_dist[pic_id].append(user_distances[(user1, user2)][0])
            users_cos_dist[pic_id].append(user_distances[(user1, user2)][1])
    
    

CPU times: user 11min 5s, sys: 7.27 s, total: 11min 12s
Wall time: 11min 8s


In [6]:
users_count

1166924

In [21]:
def distance_users(project_user,users_project, source_user, target_user):
    dict_source=users_project[source_user]
    dict_target=users_project[target_user]
    dist=0
    for pic_id in project_user:
        if pic_id in dict_source and pic_id in dict_target:
            dist=dist+0.0
        elif pic_id not in dict_source and pic_id not in dict_target:
            dist=dist+0.0
        else:
            dist=dist+1.0
    return math.sqrt(dist)      

In [26]:
def cosine_distance_users(project_user,users_project, source_user, target_user):
    dict_source=users_project[source_user]
    dict_target=users_project[target_user]
    dist=0
    dist_s= len(dict_source)
    dist_t= len(dict_source)
    for pic_id in project_user:
        if pic_id in dict_source and pic_id in dict_target:
            dist=dist+1.0          
        else:
            dist=dist+0.0
        
    return dist/(math.sqrt(dist_s*dist_t))

In [28]:

#1 - Position (i,j) is the number of pixels by user i in project j (final)
#type1 = np.zeros((users_count, projects_count))
#2 - Position (i,j) is the number of updates by user i that agree with pixels in project j.
#type2 = np.zeros((users_count, projects_count))
#3 - Position (i,j) is the number of updates by user i that in the bounding area of project j and disagree with that pixel in j. These are updates likely against the project.
#type3 = np.zeros((users_count, projects_count))
    
users_project=dict()
project_user=dict()

user_dist_proj=dict()
#with open("../data/sorted_tile_placements_proj.csv", "r") as file:
with open("../data/sorted_tile_placements_proj.csv", "r") as file:
    reader = csv.reader(file, delimiter = ",")
     # Skip first line (header row)
    next(file, None)
    # user:1, pic_id: 5, pixel: 6, pixel_color: 7
    for r in reader:
        #print(users_project)
        #print(project_user)
        #print("-------------")
        user = r[1]
        pic_id = int(r[5])
        pixel = int(r[6])
        pixel_color = int(r[7])
        if pixel==1:
            
            if pic_id not in projects_to_remove:
                if pixel_color == 1:
                    if pic_id not in project_user:
                        project_user[pic_id]=[]
                        project_user[pic_id].append(user)
                        #print("hey")
                    else:
                        if user not in project_user[pic_id]:
                            project_user[pic_id].append(user)
                        
                            
                            
                    if user not in users_project:
                        #print("hey")
                        proj_dic=dict()
                        proj_dic[pic_id]=1
                        users_project[user]=proj_dic
                    else:
                        if pic_id not in users_project[user]:
                            users_project[user][pic_id]=1
                        else:
                            users_project[user][pic_id]=users_project[user][pic_id]+1
#print(project_user) 
#print(users_project)
sample_size=10
for pic_id in project_user:
        all_users_source = list(project_user[pic_id])
        all_users_target = list(project_user[pic_id])
        
        if(sample_size<len(all_users_source)):
            all_users_source = random.sample(all_users_source, sample_size)
            all_users_target = random.sample(all_users_target, sample_size)
        
        
        avg_dist=0.0;
        for i in range(0,len(all_users_target)):
            avg_dist=avg_dist+ distance_users(project_user,users_project, all_users_source[i], all_users_target[i])
        
        user_dist_proj[pic_id]=avg_dist/sample_size
        print(user_dist_proj[pic_id])
        #print(pic_id)


2.1829159797129507
2.799284524163065
2.3221422880844313
1.5543203766865055
2.3380771057732685
2.3819864582044072
2.5968004053396796
1.4886349517372675
1.9662545694057534
1.9822587087510009
2.0938166082500502
2.177010214117348
2.798341803751984
1.9675013582531957
1.1114383155010639
1.6009967675098244
2.1776033787853604
2.0957527767097925
1.4376589663793085
1.4624973034561046
1.4617187870101171
0.3863703305156273
2.4267978754625332
1.006449510224598
1.616551208609136
1.3292528739883944
2.167047088834458
2.1394596501626415
2.882805295727564
1.8045243855508328
1.5032613887314645
1.4562660442953432
2.7137822616260663
2.546206526108746
1.8911475186413973
1.9882496571450452
1.7497731865508839
1.5945844149342396
3.1539817885622012
1.8073327514014683
2.1980334518870093
2.433604947384244
2.031461385106796
1.9125752247980716
1.8341276706445804
2.1158419378551843
2.155372180999993
1.9295923705362696
2.113477072952194
2.7888636566418703
2.0904160263422056
2.090628969044242
3.057132722470787
1.59269

2.704615399192739
2.4516953784128384
2.593932729825736
1.642757620295009
1.8074462777344227
2.6177411804141038
1.9758922011234847
1.238572731292051
2.8183591692967696
1.2545246247676256
1.2260647524952613
1.4437580654985536
1.1705529847416658
1.6482103630097824
1.9930494886488659
2.362659490727286
1.9450162202795993
2.275892201123485
2.0079539127740995
3.3503296503844338
0.8242640687119286
1.487831517751085
3.0727372339598293
1.4674861087325706
2.3468594373196985
2.0439949502709522
2.3142466813516993
2.728155850634143
1.1135047463066146
2.4241391455341708
1.6310535577717271
0.0
2.179119602824481
2.3396602205500416
2.335439646602581
2.2116671280635525
1.4830609635384557
2.337659809643324
2.224635183959092
1.7593714556303925
2.4462841798905672
2.460611647319714
1.8977045168377864
1.4765681057939963
1.7164460170806748
3.193489777947238
1.6883627011906739
1.6228766310021279
1.6496715502452404
1.9375030679963252
1.4448869930498944
2.1535855578418874
2.0951908578754717
1.6750052822528947
1.7

In [26]:
%%time 
#1 - Position (i,j) is the number of pixels by user i in project j (final)
type1 = np.zeros((users_count, projects_count))
#2 - Position (i,j) is the number of updates by user i that agree with pixels in project j.
#type2 = np.zeros((users_count, projects_count))
#3 - Position (i,j) is the number of updates by user i that in the bounding area of project j and disagree with that pixel in j. These are updates likely against the project.
#type3 = np.zeros((users_count, projects_count))
    

users_per_project = dict()
with open("../data/sorted_tile_placements_proj.csv", "r") as file:
    reader = csv.reader(file, delimiter = ",")
     # Skip first line (header row)
    next(file, None)
    # user:1, pic_id: 5, pixel: 6, pixel_color: 7
    for r in reader:
        user = r[1]
        pic_id = int(r[5])
        pixel = int(r[6])
        pixel_color = int(r[7])

        try:
            if pic_id not in projects_to_remove:

                if pic_id not in users_per_project:
                    users_per_project[pic_id] = set()

                users_per_project[pic_id].add(user)

                if pixel == 1:
                    type1[users_dict[user],project_dict[pic_id]] += 1

                #if pixel_color == 1:
                    #type2[users_dict[user],project_dict[pic_id]] += 1

                #elif pixel_color == 0:
                    #type3[users_dict[user],project_dict[pic_id]] += 1
        except Exception as e:
            print(e)
            print(pic_id)
            print(user)
            break
 
    type1 = sparse.csr_matrix(type1)
    #type2 = sparse.csr_matrix(type2)
    #type3 = sparse.csr_matrix(type3)


MemoryError: 

In [15]:
# Calculate average users per project
average_num_users = int(users_count / len(users_per_project))
print(average_num_users)

615


In [ ]:
%matplotlib inline
# plot the user distances of a single project given the pic_id
# Testing on some pic_ids
def plot_distances_icdf(pic_id):
    euc_dict = dict()
    cos_dict = dict()
    for i in range(len(users_euc_dist[pic_id])):
        euc_dict[i] = int(users_euc_dist[pic_id][i]*1000)
        cos_dict[i] = int(users_cos_dist[pic_id][i]*1000)
        
    plt.clf()

    #Computing ICDF
    count_euc = icdf(euc_dict)
    count_cos = icdf(cos_dict)
    ax = plt.subplot(111)
    ax.plot(np.arange(count_euc.shape[0])/1000, count_euc, color="red", linewidth=4, label="euclidean distance")
    ax.plot(np.arange(count_cos.shape[0])/1000, count_cos, color="blue", linewidth=4, label="cosine distance")
    ax.set_ylabel('ICDF', fontsize=30)
    ax.set_xlabel('distance between users', fontsize=30)
    ax.tick_params(labelsize=23)
    #ax.set_xlim(1, 1000000)
    ax.legend()
    ax.set_title('user distances plot for pic_id: '+str(pic_id))
    #plt.savefig(output_file_name, dpi=300, bbox_inches='tight')

plot_distances_icdf(23)

In [ ]:
#This part is not necessary

def distance_users(project_user,users_project, source_user, target_user):
	dict_source=users_project[source_user]
	dict_target=users_project[target_user]
	dist=0
	for pic_id in project_user:
		if pic_id in dict_source and pic_id in dict_target:
			dist=dist+0.0
		elif pic_id not in dict_source and pic_id not in dict_target:
			dist=dist+0.0
		else:
			dist=dist+1.0
	return math.sqrt(dist)



def user_distance_per_project(input_file_proj, projects_to_remove,sample_size):

 	"""
		#1 - Position (i,j) is the number of pixels by user i in project j (final)
		#type1 = np.zeros((users_count, projects_count))
		#We only consider type1 here

		#2 - Position (i,j) is the number of updates by user i that agree with pixels in project j.
		#type2 = np.zeros((users_count, projects_count))
		#3 - Position (i,j) is the number of updates by user i that in the bounding area of project j and disagree with that 			pixel in j. These are updates likely against the project.
		#type3 = np.zeros((users_count, projects_count))
	"""    
	users_project=dict()
	project_user=dict()

	user_dist_proj=dict()
#with open("../data/sorted_tile_placements_proj.csv", "r") as file:
	with open(input_file_proj, "r") as file:
		reader = csv.reader(file, delimiter = ",")
     # Skip first line (header row)
		next(file, None)
	# user:1, pic_id: 5, pixel: 6, pixel_color: 7
		for r in reader:
		#print(users_project)
		#print(project_user)
		#print("-------------")
			user = r[1]
			 pic_id = int(r[5])
			pixel = int(r[6])
			pixel_color = int(r[7])
			if pixel==1:
            
				if pic_id not in projects_to_remove:
					if pixel_color == 1:
						if pic_id not in project_user:
							project_user[pic_id]=[]
							project_user[pic_id].append(user)
                        #print("hey")
						else:
							if user not in project_user[pic_id]:
								project_user[pic_id].append(user)
                        
                            
                            
						if user not in users_project:
                        #print("hey")
							proj_dic=dict()
							proj_dic[pic_id]=1
							users_project[user]=proj_dic
						else:
							if pic_id not in users_project[user]:
								users_project[user][pic_id]=1
							else:
								users_project[user][pic_id]=users_project[user][pic_id]+1

	for pic_id in project_user:
		all_users_source = list(project_user[pic_id])
		all_users_target = list(project_user[pic_id])
        
		if(sample_size<len(all_users_source)):
			all_users_source = random.sample(all_users_source, sample_size)
			all_users_target = random.sample(all_users_target, sample_size)
        
        
		avg_dist=0.0;
		for i in range(0,len(all_users_target)):
			avg_dist=avg_dist+ distance_users(project_user,users_project, all_users_source[i], all_users_target[i])
        
		user_dist_proj[pic_id]=avg_dist/sample_size
        #print(user_dist_proj[pic_id])
        #print(pic_id)
	return user_dist_proj

In [9]:
from sklearn.cluster.bicluster import SpectralCoclustering

input_file_proj = "../data/sorted_tile_placements_proj.csv"
user_proj_matrix_pixel = user_project_matrix_pixel(input_file_proj, projects_to_remove)

n_clusters = 100

cocluster = SpectralCoclustering(n_clusters=n_clusters,
                                 svd_method='arpack', random_state=0)

cocluster.fit(user_proj_matrix_pixel[0])
y_cocluster = cocluster.row_labels_

x_cocluster = cocluster.column_labels_


In [ ]:
for col in x_cocluster:
    print(user_proj_matrix_pixel[2][str(col)]," in cluster ",str(col))